## Estimación de ingresos por cuidados no remunerados en la Ciudad de México

El presente script tiene como objetivo estimar los ingresos por cuidados no remunerados en la Ciudad de México. Para ello, se utilizarán los datos de la Encuesta Nacional de Ingresos y Gastos de los Hogares (ENIGH) 2022.
El objetivo es determinar el incremento en el ingreso de los hogares de la Ciudad de México si se consideran los ingresos por cuidados no remunerados. Para ello, se utilizarán los datos de la ENIGH 2022 y se estimarán los ingresos por cuidados no remunerados en la Ciudad de México. Posteriormente, se calculará el ingreso total de los hogares considerando los ingresos por cuidados no remunerados y se comparará con el ingreso total sin considerar estos ingresos.

Para el cálculo de los ingresos por cuidados no remunerados, se considerará el tiempo dedicado a los cuidados y se calculará el ingreso en función del salario mínimo de 2022, el cual fue de 172.87 pesos diarios.

In [134]:
rm(list=ls())

# Librerías ====
if(!require('pacman')) install.packages('pacman')
pacman::p_load(tidyverse, srvyr)

In [135]:
#Crear directorio para guardar los archivos
dir.create("data", showWarnings = FALSE)

In [136]:
url_basica<-"https://www.inegi.org.mx/contenidos/programas/enigh/nc/"
segunda_parte<-"microdatos/enigh"
fin1<-"_ns_"
fin2<-"_csv.zip"

In [137]:
years<-c("2022")

In [138]:
# Función genérica para descargar, descomprimir, renombrar y eliminar
procesar_archivo <- function(year, tipo, url_base, segunda_parte, fin1, fin2, carpeta_destino) {
  # Construir URL y ruta de destino
  url <- paste0(url_base, year, "/", segunda_parte, year, fin1, tipo, fin2)
  destfile <- paste0(carpeta_destino, "/", year, fin1, tipo, fin2)

  # Descargar el archivo
  download.file(url, destfile = destfile)

  # Descomprimir el archivo
  unzip(destfile, exdir = carpeta_destino)

  # Renombrar el archivo descomprimido
  file.rename(
    from = paste0(carpeta_destino, "/", tipo, ".csv"),
    to = paste0(carpeta_destino, "/", tipo, "_", year, ".csv")
  )

  # Eliminar el archivo descargado
  file.remove(destfile)
}

In [139]:
# Iterar sobre los años y procesar cada tipo de archivo

for (i in 1:length(years)) {
  procesar_archivo(years[i], "poblacion", url_basica, segunda_parte, fin1, fin2, "data")
  procesar_archivo(years[i], "concentradohogar", url_basica, segunda_parte, fin1, fin2, "data")
}

### Identificar población que se dedica a cuidados no remunerados

In [140]:
cuidadores<-read_csv("data/poblacion_2022.csv", show_col_types = FALSE)%>%
  janitor::clean_names()%>%
  #Se crea clave de entidad
  mutate(cve_ent = case_when(
    nchar(folioviv) == 9  ~ paste0("0", substr(folioviv, 1, 1)),
    nchar(folioviv) == 10 ~ substr(folioviv, 1, 2)
  ))%>%
  #Filtrar para Ciudad de México
  filter(cve_ent == "09")%>%

  #Transformar tiempos de cuidados a días para poder calcular ingresos
  mutate(
  #Transformar minutos a horas
  horas_cuidados=min_4/60,
  tiempo_cuidados=horas_cuidados+hor_4,
  #Pasar horas a días considerando que son horas semanales
    cuidados_semanales_dias=tiempo_cuidados/24,
#Calcular cuántos días fueron al trimestre
    dias_cuidados_trim=cuidados_semanales_dias*90,
 #Calcular ingreso diario considerando salario mínimo de 172.87
    ingreso_diario_cuidados=cuidados_semanales_dias*172.87,
    ingreso_cuidados_trim=ingreso_diario_cuidados*90,
    #Identificar si la persona es cuidadora
cuidador = ifelse(tiempo_cuidados > 0 | !is.na(tiempo_cuidados), 1, 0)
    )%>%
    #Filtrar cuidadores
    filter(cuidador==1)%>%
    #Seleccionar variables de interés
    select(folioviv,foliohog,tiempo_cuidados,hor_4,min_4,cuidados_semanales_dias,dias_cuidados_trim,ingreso_diario_cuidados,ingreso_cuidados_trim)%>%
    #Agrupar por por vivienda y hogar para sumar tiempo de cuidados e ingresos.
  group_by(folioviv,foliohog)%>%
    summarise(tiempo_cuidados=sum(tiempo_cuidados),
                dias_cuidados_trim=sum(dias_cuidados_trim),
                ingreso_cuidados_trim=sum(ingreso_cuidados_trim))%>%
    ungroup()
head(cuidadores)

Warning message:
"One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)"
`summarise()` has grouped output by 'folioviv'. You can override using the
`.groups` argument.


folioviv,foliohog,tiempo_cuidados,dias_cuidados_trim,ingreso_cuidados_trim
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
0900006901,1,28,105.00,18151.350
0900009201,1,9,33.75,5834.363
0900017403,1,30,112.50,19447.875
0900297701,1,10,37.50,6482.625
0900313902,1,26,97.50,16854.825
0900401202,1,96,360.00,62233.200


### Leer datos de hogares para pegar información de cuidados

In [141]:
### Leer concentrado hogar

hogares<-read_csv("data/concentradohogar_2022.csv", show_col_types = FALSE)%>%
  janitor::clean_names()%>%

 #Ordenar de mayor a menor ingreso
    arrange(desc(ing_cor))%>%
  #Se crea clave de entidad
  mutate(cve_ent = case_when(
    nchar(folioviv) == 9  ~ paste0("0", substr(folioviv, 1, 1)),
    nchar(folioviv) == 10 ~ substr(folioviv, 1, 2)
  ),
  #crear deciles
    decil = ntile(ing_cor, 10),
  #Crear quintiles
    quintil = ntile(ing_cor, 5))%>%
  # Filtrar Ciudad de México
filter(cve_ent == "09")
print(nrow(hogares))

hogares<-hogares%>%
#Pegar información de cuidadores
left_join(cuidadores, by=c("folioviv","foliohog"))%>%
#Sumar ingreso de cuidados al ingreso del hogar
mutate(ing_cor_con_cuidados=coalesce(ing_cor + ingreso_cuidados_trim, ing_cor))
print(nrow(hogares))


[1] 2585
[1] 2585


In [142]:
##Diseño nuestral
dm<-hogares%>%
as_survey_design(ids = upm,
                     strata = est_dis,
                     weights = factor)



In [143]:
#Total de hogares
print("Hogares en la Ciudad de México e ingreso corriente trimestral sin considerar cuidados y considerando estimación de ingresos por cuidados por decil, 2022")
print("Miles de pesos")
dm%>%
group_by(decil)%>%
summarise(hogares=survey_total(vartype="cv"),
            ingreso_total_sin_cuidados=survey_total(ing_cor,vartype="cv"),
            ingreso_total_con_cuidados=survey_total(ing_cor_con_cuidados,vartype="cv"))%>%
            #Ingresos en miles de pesos
mutate(ingreso_total_sin_cuidados=ingreso_total_sin_cuidados/1000,
        ingreso_total_con_cuidados=ingreso_total_con_cuidados/1000)%>%
        #Calcular variación porcentual de ingresos
mutate(variacion_pct=100*(ingreso_total_con_cuidados-ingreso_total_sin_cuidados)/ingreso_total_sin_cuidados)

[1] "Hogares en la Ciudad de México e ingreso corriente trimestral sin considerar cuidados y considerando estimación de ingresos por cuidados por decil, 2022"
[1] "Miles de pesos"


decil,hogares,hogares_cv,ingreso_total_sin_cuidados,ingreso_total_sin_cuidados_cv,ingreso_total_con_cuidados,ingreso_total_con_cuidados_cv,variacion_pct
<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,98576,0.12379155,1264257,0.13005626,1651241,0.15711173,30.609617
2,117070,0.11394414,2533417,0.11355119,3187545,0.12713734,25.819961
3,191325,0.08386196,5445346,0.08381824,6546242,0.08841450,20.217182
4,228513,0.07805405,8052209,0.07795998,10279973,0.08517980,27.666498
5,270422,0.07390807,11397930,0.07433793,13289587,0.07678280,16.596492
6,298592,0.06746130,15106918,0.06747361,17467297,0.07061117,15.624490
7,390083,0.06477733,23586140,0.06472309,27384368,0.06683649,16.103646
8,391825,0.05941874,29304112,0.05970973,32468826,0.06021924,10.799555
9,389141,0.06428477,37997705,0.06334324,41470425,0.06426986,9.139289


In [144]:
#Total de hogares
print("Hogares en la Ciudad de México e ingreso corriente trimestral sin considerar cuidados y considerando estimación de ingresos por cuidados por quintil, 2022")
print("Miles de pesos")
dm%>%
group_by(quintil)%>%
summarise(hogares=survey_total(vartype="cv"),
            ingreso_total_sin_cuidados=survey_total(ing_cor,vartype="cv"),
            ingreso_total_con_cuidados=survey_total(ing_cor_con_cuidados,vartype="cv"))%>%
            #Ingresos en miles de pesos
mutate(ingreso_total_sin_cuidados=ingreso_total_sin_cuidados/1000,
        ingreso_total_con_cuidados=ingreso_total_con_cuidados/1000)%>%
        #Calcular variación porcentual de ingresos
mutate(variacion_pct=100*(ingreso_total_con_cuidados-ingreso_total_sin_cuidados)/ingreso_total_sin_cuidados)

[1] "Hogares en la Ciudad de México e ingreso corriente trimestral sin considerar cuidados y considerando estimación de ingresos por cuidados por quintil, 2022"
[1] "Miles de pesos"


quintil,hogares,hogares_cv,ingreso_total_sin_cuidados,ingreso_total_sin_cuidados_cv,ingreso_total_con_cuidados,ingreso_total_con_cuidados_cv,variacion_pct
<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,215646,0.08604737,3797674,0.08961659,4838785,0.10000465,27.414452
2,419838,0.05829913,13497555,0.05842120,16826215,0.06493582,24.661205
3,569014,0.04758926,26504848,0.04772706,30756884,0.04959127,16.042482
4,781908,0.04432424,52890252,0.04418917,59853194,0.04489095,13.164888
5,1003624,0.04028678,170350050,0.06458015,178232445,0.06341321,4.627175
